In [1]:
import matplotlib.colors as mcolors
import matplotlib.animation as ma
import matplotlib.pyplot as plt
from itertools import product
from typing import List
import multiprocessing
import pandas as pd
import numpy as np
import numba as nb
import imageio
import sys
import os
import shutil

randomSeed = 10

if "ipykernel_launcher.py" in sys.argv[0]:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

new_cmap = mcolors.LinearSegmentedColormap.from_list(
    "new", plt.cm.jet(np.linspace(0, 1, 256)) * 0.85, N=256
)

@nb.njit
def colors_idx(phaseTheta):
    return np.floor(256 - phaseTheta / (2 * np.pi) * 256).astype(np.int32)

import seaborn as sns

sns.set_theme(font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

if os.path.exists("/opt/conda/bin/ffmpeg"):
    plt.rcParams['animation.ffmpeg_path'] = "/opt/conda/bin/ffmpeg"
else:
    plt.rcParams['animation.ffmpeg_path'] = "D:/Programs/ffmpeg/bin/ffmpeg.exe"

colors = ["#5C49AC", "#1F9BCE", "#00B5CA", "#BED45F", "#F9E03B"]
cmap = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors)
cmap_r = mcolors.LinearSegmentedColormap.from_list("my_colormap", colors[::-1])

from main import *

In [3]:
sns.set_theme(font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

def draw_mp4(model: MobileDrive, savePath: str = "./data", mp4Path: str = "./mp4", step: int = 1, earlyStop: int = None):

    targetPath = f"{savePath}/{model}.h5"
    totalPositionX = pd.read_hdf(targetPath, key="positionX")
    totalPhaseTheta = pd.read_hdf(targetPath, key="phaseTheta")
    drivePosAndPhs = pd.read_hdf(targetPath, key="drivePosAndPhs")
    TNum = totalPositionX.shape[0] // model.agentsNum
    totalPositionX = totalPositionX.values.reshape(TNum, model.agentsNum, 2)
    totalPhaseTheta = totalPhaseTheta.values.reshape(TNum, model.agentsNum)
    drivePosAndPhs = drivePosAndPhs.values.reshape(TNum, 3)
    drivePositions = drivePosAndPhs[:, :2]
    drivePhaseThetas = drivePosAndPhs[:, 2]
    if earlyStop is not None:
        totalPositionX = totalPositionX[:earlyStop]
        totalPhaseTheta = totalPhaseTheta[:earlyStop]
        TNum = earlyStop
    maxAbsPos = np.max(np.abs(totalPositionX))

    def plot_frame(i):
        pbar.update(1)
        positionX = totalPositionX[i]
        phaseTheta = totalPhaseTheta[i]
        drivePosition = drivePositions[i]
        fig.clear()
        fig.subplots_adjust(left=0.15, right=1, bottom=0.1, top=0.95)
        ax1 = plt.subplot(1, 1, 1)
        model.positionX = positionX
        model.phaseTheta = phaseTheta
        model.counts = i * model.shotsnaps
        # StateAnalysis.plot_last_state(model, ax1)
        StateAnalysis.plot_last_state(model, ax1, drivePosition=drivePosition)
        ax1.set_xlim(-maxAbsPos, maxAbsPos)
        ax1.set_ylim(-maxAbsPos, maxAbsPos)
        roundBound = np.round(maxAbsPos)
        ax1.set_xticks([-roundBound, -roundBound / 2, 0, roundBound / 2, roundBound])
        ax1.set_yticks([-roundBound, -roundBound / 2, 0, roundBound / 2, roundBound])

    frames = np.arange(0, TNum, step)
    pbar = tqdm(total=len(frames))
    fig, ax = plt.subplots(figsize=(5, 4))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=50, repeat=False)
    ani.save(f"{mp4Path}/{model}.mp4", dpi=100)
    plt.close()

    pbar.close()

In [3]:
Fs = np.linspace(0, 5, 30)
Ks = np.sort(np.concatenate([np.linspace(-1, 1, 30), [0]]))
Fs[0], Fs[3], Fs[6], Fs[12]

(0.0, 0.5172413793103449, 1.0344827586206897, 2.0689655172413794)

In [31]:
model = MobileDrive(agentsNum=500, K=1, J=0.1, F=Fs[12], savePath="./dataForMp4", randomSeed=10, dt=0.01, tqdm=True, overWrite=True)

In [32]:
model.run(5000)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [10]:
def run_model(model: MobileDrive):
    model.run(5000)

models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.01, tqdm=True)
    for K, J, F in product([1], [0.1], [Fs[0], Fs[3], Fs[6], Fs[12]])
]
# with multiprocessing.Pool(4) as p:
#     p.map(run_model, models)

In [4]:
sns.set_theme(font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

def draw_mp4s(sas: List[StateAnalysis], fileName: str, mp4Path: str = "./mp4", step: int = 1, earlyStop: int = None):
    if earlyStop is not None:
        TNum = earlyStop
    else:
        TNum = sas[0].TNum
    maxAbsPoses = [np.max(np.abs(sa.totalPositionX)) for sa in sas]

    def plot_frame(i):
        pbar.update(1)
        for sa in sas:
            sa.lookIndex = i
        fig.clear()
        for idx in range(len(sas)):
            ax = plt.subplot(rowNum, 2, idx + 1)
            sa = sas[idx]
            sa.plot_last_state(ax=ax)
            F, J, K = sa.model.F, sa.model.J, sa.model.K
            FinTitle = np.round(F) if F > 0.6 else np.round(F, 1)
            ax.set_title(f"({chr(65 + idx).lower()})  $F={FinTitle:.1f},\ J={J},\ K={K}$", fontweight="bold")
            maxPos = maxAbsPoses[idx]
            if maxPos < 1:
                ax.set_xlim(-1, 1)
                ax.set_ylim(-1, 1)
                ax.set_xticks([-1, -0.5, 0, 0.5, 1])
                ax.set_yticks([-1, -0.5, 0, 0.5, 1])
            else:
                bound = maxPos * 1.05
                roundBound = np.round(bound)
                ax.set_xlim(-bound, bound)
                ax.set_ylim(-bound, bound)
                ax.set_xticks([-roundBound, -roundBound / 2, 0, roundBound / 2, roundBound])
                ax.set_yticks([-roundBound, -roundBound / 2, 0, roundBound / 2, roundBound])
        plt.tight_layout()

    rowNum = len(sas) // 2
    fig, axs = plt.subplots(rowNum, 2, figsize=(10, rowNum * 4.2))
    axs = axs.flatten()
    frames = np.arange(0, TNum, step)
    pbar = tqdm(total=len(frames))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=50, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=100)
    plt.close()
    pbar.close()

In [34]:
Fs = np.linspace(0, 5, 30)
Ks = np.sort(np.concatenate([np.linspace(-1, 1, 30), [0]]))
models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.01, tqdm=True)
    for K, J, F in product([1], [0.1], [Fs[0], Fs[3], Fs[6], Fs[12]])
]
sas = [StateAnalysis(model) for model in models]

draw_mp4s(sas, "SS", earlyStop=200)

  0%|          | 0/200 [00:00<?, ?it/s]

In [40]:
Fs = np.linspace(0, 5, 30)
Ks = np.sort(np.concatenate([np.linspace(-1, 1, 30), [0]]))
models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.01, tqdm=True)
    for K, J, F in product([-1], [0.1], [Fs[0], Fs[3], Fs[6], Fs[12], Fs[17], Fs[29]])
]
sas = [StateAnalysis(model) for model in models]

draw_mp4s(sas, "SA", earlyStop=200)

  0%|          | 0/200 [00:00<?, ?it/s]

In [48]:
Fs = np.linspace(0, 5, 30)
Ks = np.sort(np.concatenate([np.linspace(-1, 1, 30), [0]]))
models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True)
    for K, J, F in product([Ks[13]], [1], [Fs[0], Fs[3], Fs[6], Fs[12]])
]
sas = [StateAnalysis(model) for model in models]

draw_mp4s(sas, "SPPW", earlyStop=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [50]:
Fs = np.linspace(0, 5, 30)
Ks = np.sort(np.concatenate([np.linspace(-1, 1, 30), [0]]))
models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.01, tqdm=True)
    for K, J, F in product([Ks[4]], [1], [Fs[0], Fs[3], Fs[6], Fs[17]])
]
sas = [StateAnalysis(model) for model in models]

draw_mp4s(sas, "APW", earlyStop=None)

  0%|          | 0/500 [00:00<?, ?it/s]

In [6]:
Fs = np.linspace(0, 5, 30)
Ks = np.sort(np.concatenate([np.linspace(-1, 1, 30), [0]]))
models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.01, tqdm=True)
    for K, J, F in product([0], [1], [Fs[0], 0.1, 0.2, Fs[3], Fs[6], Fs[12]])
]
sas = [StateAnalysis(model) for model in models]

draw_mp4s(sas, "SPW", earlyStop=500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
models = [
    MobileDrive(agentsNum=500, K=K, J=J, F=F, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True, overWrite=True)
    for K, J, F in product([0], [1], [0, 0.1, 0.2, 0.3, 0.4, 0.5])
]
sas = [StateAnalysis(model) for model in models]

draw_mp4s(sas, "SPW_LowF", earlyStop=None)

In [6]:
sns.set_theme(font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})

draw_mp4(model, earlyStop=100, savePath="./dataForMp4")

  0%|          | 0/100 [00:00<?, ?it/s]

In [119]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches

def draw_mp4s_phase_diagram(sas: List[StateAnalysis], fileName: str, mp4Path: str = "./mp4", 
                            step: int = 1, earlyStop: int = None):
    if earlyStop is not None:
        TNum = earlyStop
    else:
        TNum = sas[0].TNum
    wide = 0.45
    maxAbsPoses = [np.max(np.abs(sa.totalPositionX)) for sa in sas]

    def plot_frame(i):
        pbar.update(1)
        for sa in sas:
            sa.lookIndex = i
        fig.clear()
        ax = plt.subplot(1, 1, 1)
        ax.add_patch(patches.FancyArrowPatch(
            (-1, 0), (0.4, 0),
            color='black', arrowstyle='->', mutation_scale=15, linewidth=2
        ))
        ax.add_patch(patches.FancyArrowPatch(
            (0, -0.1), (0, 1),
            color='black', arrowstyle='->', mutation_scale=15, linewidth=2
        ))
        ax.add_patch(patches.FancyArrowPatch(
            (0.2, 0.6), (0, 0.6),
            color='#880015', arrowstyle='->', mutation_scale=15, linewidth=2
        ))
        ax.plot([0, -1], [0, 0.7], "k--", lw=2)
        ax.plot([0, -0.5], [0, 1], "k--", lw=2)
        axSA = inset_axes(ax, width="40%", height="60%", loc='lower left',
                  bbox_to_anchor=(0.1, 0.05, wide, wide),
                  bbox_transform=ax.transAxes)
        sas[0].plot_last_state(ax=axSA, withColorBar=False, withDriver=False, s=15)
        axSA.set_title("静态异步", fontsize=20)
        axAPW = inset_axes(ax, width="40%", height="60%", loc='lower left',
                  bbox_to_anchor=(0.15, 0.6, wide, wide),
                  bbox_transform=ax.transAxes)
        sas[1].plot_last_state(ax=axAPW, withColorBar=False, withDriver=False, s=15)
        axAPW.set_title("活跃相位波", fontsize=20)
        axSPPW = inset_axes(ax, width="40%", height="60%", loc='lower left',
                  bbox_to_anchor=(0.5, 0.6, wide, wide),
                  bbox_transform=ax.transAxes)
        sas[2].plot_last_state(ax=axSPPW, withColorBar=False, withDriver=False, s=15)
        axSPPW.set_title("分裂相位波", fontsize=20)
        axSPW = inset_axes(ax, width="40%", height="60%", loc='lower left',
                  bbox_to_anchor=(0.75, 0.6, wide, wide),
                  bbox_transform=ax.transAxes)
        sas[3].plot_last_state(ax=axSPW, withColorBar=False, withDriver=False, s=15)
        axSPW.set_title("静态相位波", fontsize=20)
        axSS = inset_axes(ax, width="40%", height="60%", loc='lower left',
                  bbox_to_anchor=(0.75, 0.05, wide, wide),
                  bbox_transform=ax.transAxes)
        sas[4].plot_last_state(ax=axSS, withColorBar=False, withDriver=False, s=15)
        axSS.set_title("静态同步", fontsize=20)

        for idx, subAx in enumerate([axSA, axAPW, axSPPW, axSPW, axSS]):
            subAx.set_xticks([])
            subAx.set_yticks([])
            if maxAbsPoses[idx] < 1:
                subAx.set_xlim(-1, 1)
                subAx.set_ylim(-1, 1)
            else:
                bound = maxAbsPoses[idx] * 1.05
                roundBound = np.round(bound)
                subAx.set_xlim(-bound, bound)
                subAx.set_ylim(-bound, bound)
        
        ax.set_xlim(-1, 0.4)
        ax.set_ylim(-0.02, 1.05)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel(r"K", fontsize=20)
        ax.set_ylabel(r"J", rotation=0, fontsize=20, labelpad=-480)
        
        plt.tight_layout()

    rowNum = len(sas) // 2
    fig, axs = plt.subplots(rowNum, 2, figsize=(10, 7))
    axs = axs.flatten()
    frames = np.arange(0, TNum, step)
    pbar = tqdm(total=len(frames))
    ani = ma.FuncAnimation(fig, plot_frame, frames=frames, interval=50, repeat=False)
    ani.save(f"{mp4Path}/{fileName}.mp4", dpi=100)
    plt.close()
    pbar.close()

In [120]:
import matplotlib.font_manager as fm
fontpath = 'FandolSong-Regular.otf'
font = fm.FontProperties(fname=fontpath)
sns.set_theme(
    font=font.get_name(),
    font_scale=1.1, rc={
    'figure.figsize': (6, 5),
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'grid.color': '#dddddd',
    'grid.linewidth': 0.5,
    "lines.linewidth": 1.5,
    'text.color': '#000000',
    'figure.titleweight': "bold",
    'xtick.color': '#000000',
    'ytick.color': '#000000'
})
plt.rcParams['axes.unicode_minus'] = False
models = [
    MobileDrive(agentsNum=500, K=-1, J=0.1, F=0, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True, overWrite=True),
    MobileDrive(agentsNum=500, K=-0.724, J=1, F=0, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True, overWrite=True),
    MobileDrive(agentsNum=500, K=-0.1, J=1, F=0, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True, overWrite=True),
    MobileDrive(agentsNum=500, K=0, J=1, F=0, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True, overWrite=True),
    MobileDrive(agentsNum=500, K=1, J=0.1, F=0, savePath="./dataForMp4", randomSeed=10, dt=0.03, tqdm=True, overWrite=True),
]
sas = [StateAnalysis(model) for model in models]
draw_mp4s_phase_diagram(sas, fileName="PhaseDiagram", earlyStop=None)

  0%|          | 0/500 [00:00<?, ?it/s]